In [ ]:
import os
os.chdir('/home/garskova-d/metrika-analytics')

from metr_utils.databases import clickhouse as ch
from metr_utils.databases import yql_db
from metr_utils.databases import yt_db
from metr_utils.databases import mysql_db
from metr_utils.databases import psql_db
from metr_utils.configs import options
from metr_utils.configs import get_vault_data
from metr_utils import generate_query
from metr_utils import plotly_graphs as pplot
from metr_utils import pandas_helpers as ph
from metr_utils import blackbox as bb
import pandas as pd
import numpy as np
from io import StringIO
from datetime import datetime, timedelta
import time
import builtins
builtins.opts = options.args(get_robot_passes=True)

In [ ]:
import warnings
import sys
warnings.simplefilter(action='ignore')
sys.path.append("../")
from fbprophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
import matplotlib.pyplot as plt
import math
import itertools
import tqdm
import pandas
import pandas as pd
import numpy
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

pylab.rcParams.update(params)

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

import multiprocessing
import time
import concurrent.futures

from string import Template
import csv
import json
import tqdm
from os.path import exists

import requests
import re

! mkdir raw_predict

In [ ]:
def get_dates(today_str, n_periods):

    daily_today = pd.to_datetime(today_str)
    daily_end_date = pd.to_datetime(today_str) - pd.Timedelta(days = n_periods)
    daily_end_date_str = daily_end_date.strftime("%Y-%m-%d")
    daily_month_ago = daily_end_date - pd.Timedelta(days = 30)
    daily_3_months_ago = daily_end_date - pd.Timedelta(days = 90)
    daily_1_year_ago = daily_end_date - pd.Timedelta(days = 365)
    daily_1_year_ago_str = daily_1_year_ago.strftime("%Y-%m-%d")

    n_periods_future = 90
    daily_month_ago_future = daily_today - pd.Timedelta(days = 30)
    daily_3_months_ago_future = daily_today - pd.Timedelta(days = 90)
    
    daily_3_years_ago_future = daily_today - pd.Timedelta(days = 365*3)
    daily_3_years_ago_future_str =  daily_3_years_ago_future.strftime("%Y-%m-%d")

    start_of_today_week = daily_today.to_period('W').start_time
    
    return daily_today, daily_end_date, daily_end_date_str, daily_month_ago, daily_3_months_ago, daily_1_year_ago, \
            daily_1_year_ago_str, n_periods_future, daily_month_ago_future, daily_3_months_ago_future, \
            daily_3_years_ago_future, daily_3_years_ago_future_str, start_of_today_week

In [ ]:
def get_active_counters_in_interface(months_ago = 6):
    
    global today_str
    
    active = f'''
    select counter_id as CounterID, uniq(watch_id) as hits, uniq(date) as dates

    from (
    select
            WatchID as watch_id,
            UserID as user_id,
            EventDate as date,
            EventTime as datetime,
            PassportUserID as passport_user_id,
            URLPathHierarchy(cutQueryStringAndFragment(URL)) as hierarchy,
            arrayMap(x -> if(x like '%/', substring(x, 1, length(x) - 1), x), URLPathHierarchy(cutQueryStringAndFragment(URL))) as hierarchy_filt,
            replaceAll(hierarchy_filt[1], '//', '/') as main_url_path,
            replaceAll(if(length(hierarchy_filt) = 1, hierarchy_filt[1], hierarchy_filt[2]), '//', '/') as url_path,
            extract(if(extractURLParameter(URL, 'id') = '', extractURLParameter(URL, 'ids'), extractURLParameter(URL, 'id')), '[0-9]+') as counter_id
        from hits_all
        where EventDate >= toDate('{today_str}')-30*{months_ago}
            and EventDate <= '{today_str}'
            and CounterID = 24226447
            and not DontCountHits
            and not IsRobot
            and url_path not in ('/inpage/visor-proto', '/inpage/visor-player', '/inpage')
            and main_url_path in ('/stat', '/inpage', '/publishers', '/markedphones', '/legacy')
            and counter_id != ''
        settings max_rows_in_join = 0, max_bytes_in_join = 0, max_memory_usage = 100000000000)
    group by counter_id
    settings max_rows_in_join = 0, max_bytes_in_join = 0, max_memory_usage = 100000000000
    format TSVWithNames
    '''
    df_counters_active = ch.get_df(active)
    df_counters_active.CounterID = df_counters_active.CounterID.astype(int)
    df_counters_active = df_counters_active[df_counters_active.CounterID > 0]
    
    return df_counters_active

In [ ]:
#! mkdir counters_daily_fact
def get_counter_visits(counter_id, add_additional_days = True):
    
    global daily_today
    global today_str
    global daily_3_years_ago_future
    global daily_3_years_ago_future_str
    
    first_date_str = daily_3_years_ago_future_str
    
    filename = f'counters_daily_fact/daily_fact_{counter_id}.csv'
    
    daily_fact_сounter = pd.DataFrame()
    
    df_fact = pd.DataFrame()
    
    if add_additional_days == False and exists(filename):
        pass
    
    else:

        if exists(filename):

            daily_fact_сounter = pd.read_csv(filename)
            daily_fact_сounter['ds'] = pd.to_datetime(daily_fact_сounter['ds'])
            first_date = daily_fact_сounter.ds.max() + pd.Timedelta(days = 1)
            first_date_str = first_date.strftime("%Y-%m-%d")

        query = f'''
        SELECT StartDate as ds, sum(Sign) AS y 
        FROM visits_all 
        WHERE CounterID = {counter_id}
            and StartDate >= '{first_date_str}'
            and StartDate <= '{today_str}'
            and ClientIPNetwork != 'YANDEX_SERVER' 
            and IsRobot = 0 
        GROUP BY ds
        HAVING y >= 0
        FORMAT TabSeparatedWithNames
        '''
        for i in range(0, 3):
            try:
                df_fact = ch.get_df(query=query)
                break
            except:
                time.sleep(5)
            
        try:
            df_fact['ds'] = pd.to_datetime(df_fact['ds'])
        except:
            pass

        if exists(filename):
            df_fact = daily_fact_сounter.append(df_fact).reset_index(drop = True)

        true_dates = pd.DataFrame(pd.date_range(df_fact.ds.min(), df_fact.ds.max()), columns = ['ds'])
        df_fact = true_dates.merge(df_fact, how = 'left', left_on = 'ds', right_on = 'ds').fillna(0)
        df_fact['y'] = df_fact['y'].astype(int)

        if df_fact.shape[0] > 0:
            df_fact.to_csv(filename, index = False, sep = ',')
            #print('add')

In [ ]:
def mape(A, F): 
    return np.mean(np.abs((A - F) / A)) * 100
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))
def smape_median(A, F):
    return 100 * (2 * np.abs(F - A) / (np.abs(A) + np.abs(F))).median()
def rmse(A, F):
    return np.sqrt(mse(A, F))

def smape_sum(A, F):
    return (2 * np.abs(F - A) / (np.abs(A) + np.abs(F))).sum()

In [ ]:
def get_current_holidays():
    df_custom_holidays = pd.read_csv('~/arcadia/statbox/jam/actions/outer-action/analytics_keymetrics/research-3455/action/analyticskeymetrics/forecasting/models/holidays_list.csv')
    holidays_list = df_custom_holidays.ds.tolist()
    holidays_list_full = []
    for day in df_custom_holidays.iterrows():
        start = pd.to_datetime(day[1].ds) + timedelta(days=day[1].lower_window)
        end = pd.to_datetime(day[1].ds) + timedelta(days=day[1].upper_window)
        holidays_list_full = holidays_list_full + list(pd.date_range(start, end))
    return df_custom_holidays, holidays_list_full

In [ ]:
def make_prediction(df_train,
                    df_fact, 
                    capacity, 
                    freq, 
                    n_periods, 
                    date_end, 
                    n_periods_future,
                    off_weekly = False, 
                    holidays_prior_scale = 8):
    
    global df_custom_holidays
    global holidays_list_full
    
    # firstly we run model on train data, predict period that already happened in order to evaluate the result:
    
    train_df = df_train.copy() 
    
    # obligatory columns for using logistic type of growth
    # these restrictions are only for trend-compopent, not for seasonality:
    
    train_df['cap'] = df_train.y.max()*capacity # ceil is 1.1*max_visits, where max_visits is maximum visits of last 3 years
    train_df['floor'] = 0 # floor for trend is 0
 
    weekly_seasonality = 'auto' # add weekly_seasonality for everyone
    
    if off_weekly: # we turn off weekly seasonality on holidays
        
        weekly_seasonality = False
        train_df['on_weekly'] = ~train_df.ds.isin(holidays_list_full) # special "seasonality" condition for holidays perios

    all_params = ['additive', 'multiplicative'] # for future grid of best seasonlity type mode
    prophet_errors = dict()
    prophet_forecasts = dict()
    prophet_featutes = {'additive':{}, 'multiplicative':{}}
    
    if train_df.shape[0] >= 365:  # add yearly seasonality if enough data (historical points more than 365)
        yearly_seasonality = True
    else:
        yearly_seasonality = 'auto' # if historical points less than 365 let prophet reveal yearly sesonality automatically 
    
    # sometimes prophet optimization failing and it results in fit problem. It may happens only for one type of 
    # seasonality. For example, only for additive sesonality mode. Therefore we trying to fit multiplicative mode. If it also
    # fails we exclude counter from prediction
    
    fit_errors = 0 # counter for fit problem. 1 - only sesonality_mode failed. 2 - both failed (exclude from prediction)

    for param in all_params:
        
        m = Prophet(seasonality_mode = param, 
                    growth = 'logistic', 
                    holidays = df_custom_holidays, 
                    holidays_prior_scale = holidays_prior_scale, 
                    weekly_seasonality=weekly_seasonality, 
                    yearly_seasonality = yearly_seasonality)

        if off_weekly: # overwrite weekly seasonality if it holiday 
            m.add_seasonality(name='weekly', period=7, fourier_order=3, condition_name='on_weekly')
    
        try:
            m.fit(train_df)
            
        except RuntimeError: # fit problem handling
            fit_errors += 1
            continue # swith to next seasonality mode if one failed               

        future = m.make_future_dataframe(periods=n_periods, freq=freq) # freq is always daily
        future['cap'] = df_train.y.max()*capacity # obligatory columns for using logistic type of growth 
        future['floor'] = 0 # obligatory columns for using logistic type of growth
        if off_weekly: # we turn off weekly seasonality on holidays
            future['on_weekly'] = ~future.ds.isin(holidays_list_full)
        forecast = m.predict(future)
        
        # our forecast can be < 0 due to seasonality compopnent (trend restictions are taken into consideration above by 
        # 'cap' column. After final summarizing of trend+seasonality+holidays components we clip all negative values to 0:
        
        forecast['yhat_final'] = forecast['yhat'].clip(lower = 0) 
        forecast['yhat_final'] = forecast['yhat_final'].apply(lambda x: round(x, 0)) # visits are only integers, not float
        
        # write forecast and its rmses for additive and mupliplicative seasonality modes in order to choose the best one later
        
        prophet_forecasts[param] = forecast
        prophet_errors[param] = rmse(df_fact[df_fact['ds']>date_end]['y'], forecast[forecast['ds']>date_end]['yhat_final'])
        for el in features:
            prophet_featutes[param][el] = getattr(m, el) # information about model
     
    if fit_errors == 2: # skip counter if prophet wasn't able to fit model eventually
        return models_info

    df_errors_tmp = pd.DataFrame(prophet_errors, index = ['prophet']).transpose()
    
    if df_errors_tmp.shape[0] != 0: 
        
        # choose the best model (additive or multiplicative) by rmse and write it as best_model:
        
        best_prophet_model = df_errors_tmp\
                            [df_errors_tmp['prophet']==df_errors_tmp['prophet'].min()].reset_index()['index'].values[0]
        forecast = prophet_forecasts[best_prophet_model]

        error_mape = 0 # mape is useful for post-analysis
        error_mape = mape(df_fact[df_fact['ds']>date_end]['y'], forecast[forecast['ds']>date_end]['yhat_final'])
        
        # add forecast to dataframe with facts:
        
        df_fact['prophet'] = np.nan 
        df_fact['prophet'].iloc[-n_periods:] = np.array(forecast[forecast['ds']>date_end]['yhat_final'])
        
        # models_info contains all the information on model:

        models_info[counter_id] = {'fact': df_fact, 'forecast':forecast,
                                   'prophet_features':prophet_featutes[best_prophet_model],
                                   'error_prophet':df_errors_tmp\
                                           [df_errors_tmp['prophet']==df_errors_tmp.prophet.min()].prophet.values[0],
                                   'mape':error_mape}

    #print(models_info[counter_id]['prophet_features'])

    scale = {'D':'D', 'W-MON':'W-MON', 'MS':'MS'} # freq is always daily, it's old code
    
    # dates for graph should also include zero points so we add them for post-analysis
    
    df_dates_for_graph = pd.DataFrame(pd.date_range(df_fact.ds.min(), df_fact.ds.max(), freq = scale[freq]), columns = ['ds'])
    df_dates_for_graph = df_dates_for_graph.merge(df_fact, how = 'left')
    df_dates_for_graph.y = df_dates_for_graph.y.fillna(0).astype(int)
    
    #print(pplot.one_df(df_dates_for_graph[['ds', 'y', 'prophet']].set_index('ds')))
    
    # next we fit model with best params and make forecast on real period:

    train_df = df_fact[['ds','y']].copy()
    train_df['cap'] = df_train.y.max()*capacity
    train_df['floor'] = 0
    
    if train_df.shape[0] >= 365:  # Add yearly seasonality if enough data
        yearly_seasonality = True
    else:
        yearly_seasonality = 'auto'
        
    weekly_seasonality = 'auto'
    if off_weekly: 
        weekly_seasonality = False
        train_df['on_weekly'] = ~train_df.ds.isin(holidays_list_full)  

    m = Prophet(seasonality_mode = best_prophet_model,
                growth = 'logistic',
                holidays = df_custom_holidays,
                holidays_prior_scale = holidays_prior_scale,
                weekly_seasonality=weekly_seasonality,
                yearly_seasonality = yearly_seasonality)

    if off_weekly:
        m.add_seasonality(name='weekly', period=7, fourier_order=3, condition_name='on_weekly')
    
    try:
        m.fit(train_df)

    except RuntimeError:
        print('fit problem')

    future = m.make_future_dataframe(periods=n_periods_future, freq=freq)
    future['cap'] = df_train.y.max()*capacity
    future['floor'] = 0
    if off_weekly:
            future['on_weekly'] = ~future.ds.isin(holidays_list_full)
    forecast = m.predict(future)
    
    forecast['forecast'] = forecast['yhat']
    forecast['forecast_upper'] = forecast['yhat_upper']
    forecast['forecast_lower'] = forecast['yhat_lower']
    
    df_fact['yhat'] = np.nan
    df_fact['yhat_lower'] = np.nan
    df_fact['yhat_upper'] = np.nan
    df_fact = df_fact.append(forecast[forecast['ds']>daily_today][['ds','yhat', 'yhat_lower', 'yhat_upper']])
    df_fact = df_fact.merge(forecast[['ds','forecast', 'forecast_upper', 'forecast_lower']], 
                            left_on = 'ds', right_on = 'ds', how = 'left')

    df_dates_for_graph = pd.DataFrame(pd.date_range(df_fact.ds.min(), df_fact.ds.max() + pd.Timedelta(days = n_periods_future), 
                                                                                                      freq = scale[freq]), 
                                                    columns = ['ds'])
    df_dates_for_graph = df_dates_for_graph.merge(df_fact, how = 'left')
    df_dates_for_graph.y = df_dates_for_graph.y.fillna(0).astype(int)
    
    #pplot.one_df(df_fact[['ds','y','yhat_final', 'yhat_lower_final', 'yhat_upper_final']].set_index('ds'))
    
    df_fact['future'] = (df_fact['ds']>daily_today).astype(int)
    models_info[counter_id]['new_predict'] = df_fact

    return models_info

In [ ]:
import requests
from ticket_parser2_py3.api import v1 as tp2_api
import vault_client
import getpass

def get_passport_secret():
    curr_user = getpass.getuser()
    bot_name = None
    if curr_user == 'airflow':
        bot_name = 'robot-conv-main'
    yav = vault_client.instances.Production(rsa_login=bot_name)
    # Версии нужно обновлять руками
    passport_secret = yav.get_version('ver-01f1yyja32kmy44n0m49qkvr4n')['value']['client_secret']
    # https://yav.yandex-team.ru/secret/sec-01dvn9t18n723mw4dvfne8r1fm/explore/versions
    return passport_secret

class TVM(object):
    def __init__(self, self_client_id, self_secret, destinations, secret_key=None):
        """
        Args:
            self_client_id (int): ID своего приложения, которое будет ходить с TVM в удаленный сервис
            self_secret (str|dict): TVM секрет приложения
            destinations (dict): Словарь {alias_name: service_id}, куда приложение будет ходить.
        """
        self.settings = tp2_api.TvmApiClientSettings(
            self_client_id=self_client_id,
            self_secret=self_secret,
            dsts=destinations,
        )
        self.client = tp2_api.TvmClient(self.settings)

def get_tvm_client(secret):
    if not secret:
        secret = get_passport_secret()
    tvm = TVM(
        self_client_id=2017315,
        self_secret=secret,
        destinations={'metrika_api_test': 2000269, 'metrika_api': 2000270},
        secret_key='client_secret',
    )
    return tvm

In [ ]:
def upload(table, content, host):
    '''Uploads data to table in ClickHouse'''
    query_dict = {
        'query': 'INSERT INTO ' + table + ' FORMAT TabSeparatedWithNames '
    }
    r = requests.post(host, data=content.encode('utf-8'), params=query_dict, auth=(options.args(get_robot_passes=True).ch_user, options.args(get_robot_passes=True).ch_pass))
    result = r.text
    if r.status_code == 200:
        return result
    else:
        raise ValueError(r.text)

def upload_df(table, df, host):
    return upload(table, df.to_csv(index=False, sep = '\t'), host=host)

## Main calculation

In [ ]:
today_str = '2022-07-25'
n_periods = 90

In [ ]:
daily_today, daily_end_date, daily_end_date_str, daily_month_ago, daily_3_months_ago, daily_1_year_ago, daily_1_year_ago_str, \
n_periods_future, daily_month_ago_future, daily_3_months_ago_future, daily_3_years_ago_future, daily_3_years_ago_future_str, \
start_of_today_week = get_dates(today_str, n_periods)

In [ ]:
features = ['growth',
'holidays_prior_scale',
'n_changepoints',
'changepoint_range',
'yearly_seasonality',
'weekly_seasonality',
'daily_seasonality',
'seasonality_mode',
'seasonality_prior_scale',
'changepoint_prior_scale',
'holidays_prior_scale',
'interval_width',
'uncertainty_samples',
'y_scale',
'logistic_floor',
't_scale',
'seasonalities',
'country_holidays',
'growth',
'component_modes']

In [ ]:
df_custom_holidays, holidays_list_full = get_current_holidays()

### Отбор счетчиков с нуля - не запускать))

In [ ]:
df_counters_active = get_active_counters_in_interface()

In [ ]:
#df_counters_active.to_csv('active_counters.csv', index = False, sep = ',')
#df_counters_active.shape[0]

In [ ]:
table = 'garskova.active_counters_in_interface'
host = 'http://mtchvip01m.metrika.yandex.net:8123/'

drop_table = '''drop TABLE if exists {table}'''.format(table=table)

create_table = ''' CREATE TABLE {table} (CounterID UInt32, hits UInt64, dates UInt64) ENGINE = Log; '''.format(table = table)

data = ch.get_df(query=drop_table, raw = True)
data = ch.get_df(query=create_table, raw = True)

upload_df(table, df_counters_active, host)

In [ ]:
counters_dfs = pd.DataFrame({'CounterID':[]})

for i in tqdm.tqdm(range(0, 10)):

    query = f'''
    select a.CounterID as CounterID from 

    (select distinct CounterID
    from (SELECT CounterID, StartDate as ds, sum(Sign) as y
    FROM visits_all
    WHERE StartDate <= toDate('{today_str}')
            and ClientIPNetwork != 'YANDEX_SERVER' 
            and IsRobot = 0
            and CounterID global in (select CounterID from garskova.active_counters_in_interface)
            and intHash32(CounterID) % 10 = {i}
    group by CounterID, StartDate)
    group by CounterID
    ) as a

    inner join 

    (SELECT CounterID, sum(Sign) as y
    FROM visits_all
    WHERE StartDate >= toDate('{today_str}') - 30 and StartDate <= toDate('{today_str}')
            and ClientIPNetwork != 'YANDEX_SERVER' 
            and IsRobot = 0
            and CounterID global in (select CounterID from garskova.active_counters_in_interface)
            and intHash32(CounterID) % 10 = {i}
    group by CounterID
    having y > 0
    ) as b

    on a.CounterID = b.CounterID

    inner join

    (SELECT CounterID, sum(Sign) as y
    FROM visits_all
    WHERE StartDate <= toDate('{daily_end_date_str}')
            and ClientIPNetwork != 'YANDEX_SERVER' 
            and IsRobot = 0
            and CounterID global in (select CounterID from garskova.active_counters_in_interface)
            and intHash32(CounterID) % 10 = {i}
    group by CounterID
    having y > 0
    ) as c

    on a.CounterID = c.CounterID

    FORMAT TabSeparatedWithNames

    '''
    df_counters_sample = ch.get_df(query)
    counters_dfs = counters_dfs.append(df_counters_sample)

In [ ]:
counters_dfs.CounterID = counters_dfs.CounterID.astype(int)

In [ ]:
counters_dfs.to_csv('counters_active_and_filtered.csv', index = False, sep = ',')

In [ ]:
# pd.Series(counters).to_csv('counters.csv', index = False, sep = ',')

### Отбор счетчиков из предыдущей итерации

In [ ]:
counters = pd.read_csv('counters_active_and_filtered.csv')
counters = counters.CounterID.to_list()

In [ ]:
len(counters)

In [ ]:
# counters_exp = pd.Series(counters[0:36000])
# counters_exp.to_csv('counters_exp.csv', index = False, sep = ',')

In [ ]:
counters = pd.read_csv('counters_exp.csv')['0'].to_list()

In [ ]:
len(counters)

In [ ]:
counters = counters[0:14000]

In [ ]:
start_time = time.perf_counter()
daily_fact = dict()

with concurrent.futures.ThreadPoolExecutor(max_workers = 2) as executor:
    futures = []
    for counter_id in counters[:]:
        try:
            futures.append(
                executor.submit(
                    get_counter_visits, counter_id=counter_id, add_additional_days = True
                )
            )
        except:
            print('error')
            
    #for counter_id, df in tqdm.tqdm(map(lambda x: x.result(), futures)):
    #    pass
    for el in tqdm.tqdm(futures):
        el.result()
finish_time = time.perf_counter()
print(f"Program finished in {finish_time-start_time} seconds")

In [ ]:
counters[1701:1704]

In [ ]:
#проверочка
filename = f'counters_daily_fact/daily_fact_87341212.csv'
daily_fact_сounter = pd.read_csv(filename)
daily_fact_сounter.tail(60)

In [ ]:
#! mkdir counters_model_info

np.random.seed(seed=42)

# РАСКОММЕНТИТЬ ЕСЛИ НАДО

models_info = dict()
result = dict()
result_scales = {'fact_daily':{},'fact_weekly':{}, 'fact_monthly':{}}
good_daily_counters = []

for counter_id in tqdm.tqdm(counters):
    #print(counter_id)
    
    #df_fact = get_counter_visits(counter_id)
    #df_fact = daily_fact[counter_id]
    
    if counter_id not in result_scales['fact_daily'].keys():
#         and daily_today not in final_result_scales['fact_daily'][counter_id]\
#         [~final_result_scales['fact_daily'][counter_id].y.isna()].ds.to_list()
        
        filename = f'counters_daily_fact/daily_fact_{counter_id}.csv'
        df_fact = pd.read_csv(filename)
        df_fact['ds'] = pd.to_datetime(df_fact['ds'])
        df_fact = df_fact[df_fact['ds'] >= daily_3_years_ago_future]

        df_train = df_fact[df_fact['ds']<=daily_end_date]
        df_predict = df_fact[df_fact['ds']>daily_end_date]

        true_dates = pd.DataFrame(pd.date_range(daily_end_date, daily_today), columns = ['date'])
        df_true_dates = true_dates.merge(df_fact[df_fact['ds']>=daily_end_date], how = 'left', left_on = 'date', right_on = 'ds')[['date', 'y']].fillna(0).rename(columns = {'date':'ds'})
        df_fact = df_fact[df_fact['ds']<daily_end_date].append(df_true_dates).reset_index(drop = True)
        df_fact.y = df_fact.y.astype(int)

        if df_train[df_train['ds']==daily_end_date].shape[0] == 0:
            #print('add extra day')
            df_train = df_train.append(pd.DataFrame({'ds':daily_end_date, 'y':[0]})).reset_index(drop = True).sort_values('ds')

        if df_train.shape[0] <= 30:
            pass
            #print('too little history observations for any forecasts:', df_train.shape[0], 'days')
        else:
            result = make_prediction(df_train, 
                                     df_fact, 
                                     capacity = 1.1, 
                                     freq = 'D', 
                                     n_periods = n_periods, 
                                     date_end = daily_end_date, 
                                     off_weekly = True,
                                     holidays_prior_scale = 8, 
                                     n_periods_future = n_periods_future)

            if counter_id not in result:
                continue

            df_daily = result[counter_id]['new_predict'].copy()
            daily_median = df_daily[(df_daily['ds']>=daily_month_ago_future) & (df_daily.ds <= daily_today)].y.median()
            result_scales['fact_daily'][counter_id] = df_daily
            df_daily.to_csv(f'counters_model_info/{counter_id}.csv', index = False, sep = ',')

            if daily_median <= 10:
                #print('too little data for daily forecast:', daily_median)
                del result[counter_id]
            else:
                good_daily_counters.append(counter_id)


final_result_daily = dict()
final_result_scales = dict()
final_result_daily = result.copy()
final_result_scales = result_scales.copy()

In [ ]:
len(list(result_scales['fact_daily'].keys()))

In [ ]:
# final_result_daily = dict()
# final_result_scales = dict()
# final_result_daily = result.copy()
# final_result_scales = result_scales.copy()

In [ ]:
daily_today in final_result_scales['fact_daily'][49434271][~final_result_scales['fact_daily'][49434271].y.isna()].ds.to_list()

In [ ]:
len(final_result_daily.keys())

In [ ]:
final_result_daily[85546441]['new_predict'].tail(93)

In [ ]:
pd.Series(list(final_result_scales['fact_daily'].keys())).to_csv('counters_fact_daily_exp_v6.csv', index = False, sep = ',')

#### нормализируем настоящий прогноз

In [ ]:
def normalize_daily_predictions(daily_today, final_result_scales, n_days):
    final_result_scales['new_predict_with_normalization'] = dict()
    last_fact_date = daily_today
    n = n_days
    start_date = last_fact_date - pd.to_timedelta('%s days' % (n-1))
    for counter_id in list(final_result_scales['fact_daily'].keys())[:]:
        df = final_result_scales['fact_daily'][counter_id][['ds','y','prophet', 'yhat', 'yhat_upper','yhat_lower',
                                                                          'forecast', 'forecast_lower', 'forecast_upper', ]]

        fact_ag = df.set_index('ds').loc[start_date:last_fact_date, 'y'].mean()
        forecast_agg = df.set_index('ds').loc[start_date:last_fact_date, 'forecast'].mean()
        fact_forecast_diff = forecast_agg - fact_ag

        df['yhat_new'] = df['yhat'] - fact_forecast_diff
        df['yhat_upper_new'] = df['yhat_upper'] - fact_forecast_diff
        df['yhat_lower_new'] = df['yhat_lower'] - fact_forecast_diff

        df['yhat'] = df['yhat'].apply(lambda x: round(x, 0))
        df['yhat_new'] = df['yhat_new'].apply(lambda x: round(x, 0))
        df['yhat_upper_new'] = df['yhat_upper_new'].apply(lambda x: round(x, 0))
        df['yhat_lower_new'] = df['yhat_lower_new'].apply(lambda x: round(x, 0))

        df = df.set_index('ds')
        df = df.clip(0)

        #pplot.one_df(df[['y','yhat_new','yhat']])
        #final_result_daily[counter_id]['new_predict_with_normalization'] = dict()
        #final_result_daily[counter_id]['new_predict_with_normalization'] = df.clip(0).reset_index()
        final_result_scales['new_predict_with_normalization'][counter_id] = dict()
        final_result_scales['new_predict_with_normalization'][counter_id] = df.clip(0).reset_index()
    return final_result_scales

In [ ]:
final_result_scales = normalize_daily_predictions(daily_today,
                                                  final_result_scales, 
                                                  n_days = 7)

In [ ]:
for counter_id in list(final_result_daily.keys())[7111:7119]:
    pplot.one_df(final_result_scales['new_predict_with_normalization'][counter_id][['ds','y', 'yhat_new']].set_index('ds'))

In [ ]:
final_result_scales['new_predict_with_normalization'][37512210].tail(92)

#### Убираем плохие прогнозы: свалившиеся в ноль. Убираем счетчики, у которых мало данных за последние 30 дней

In [ ]:
def filter_counters(counters_list, scale):
    
    global daily_month_ago_future
    global daily_today
    
    counters = []
    counters_bad = []
    counters_bad_2 = []
    for counter_id in counters_list[:]:
        if scale == 'day':
            df_try = final_result_scales['new_predict_with_normalization'][counter_id][['ds','y', 'yhat', 'yhat_new']]
            if (df_try.yhat_new==0).sum()/(df_try.yhat_new).count()>=0.5 or sum(df_try.iloc[-10:,:]['yhat_new']==0) == 10:
                counters_bad.append(counter_id)
            elif df_try.set_index('ds').loc[daily_month_ago_future:daily_today,:'y'].median().values[0] <= 10:
                counters_bad_2.append(counter_id)
            else:
                counters.append(counter_id)
        if scale == 'week':
            df_try = final_result_scales['fact_weekly'][counter_id][['ds','y','yhat_new']]
            if (df_try[df_try['ds'] > daily_today].yhat_new==0).sum()/(df_try[df_try['ds'] > daily_today].yhat_new).count()>=0.5 or sum(df_try.iloc[-3:,:]['yhat_new']==0) == 3:
                counters_bad.append(counter_id)
            elif df_try.set_index('ds').loc[daily_month_ago_future:daily_today,:'y'].median().values[0] <= 10:
                counters_bad_2.append(counter_id)
            else:
                counters.append(counter_id)
            
    return counters, counters_bad, counters_bad_2

In [ ]:
counters_for_daily, counters_bad_zeros, counters_bad_median = filter_counters(list(final_result_daily.keys()), scale = 'day')

In [ ]:
print(len(counters_for_daily), len(counters_bad_zeros), len(counters_bad_median))

#### Прогнозы неделя, месяц

In [ ]:
def weekly_monthly_predict(counters_for_weekly_monthly_predicts, print_info = False):
    for counter_id in counters_for_weekly_monthly_predicts:
        #df_daily = final_result_daily['new_predict_with_normalization'][counter_id]
        df_daily = final_result_scales['new_predict_with_normalization'][counter_id].copy()
        df_tmp_weekly = df_daily.copy()
        df_tmp_monthly = df_daily.copy()

        #print('weekly')

        start_of_today_week = daily_today.to_period('W').start_time
        end_of_today_week = pd.to_datetime(daily_today.to_period('W').end_time.strftime('%Y-%m-%d'))
        last_day_of_forecast = df_tmp_weekly.ds.max()
        start_of_last_week = last_day_of_forecast.to_period('W').start_time
        end_of_last_week = pd.to_datetime(last_day_of_forecast.to_period('W').end_time.strftime('%Y-%m-%d'))

        if daily_today != end_of_today_week:  
            df_tmp_weekly = df_tmp_weekly.set_index('ds')
            df_tmp_weekly.loc[start_of_today_week:daily_today, 'yhat_new'] = df_tmp_weekly.loc[start_of_today_week:daily_today, 'y']
            df_tmp_weekly = df_tmp_weekly.reset_index()

        df_tmp_weekly['ds'] = df_tmp_weekly['ds'].dt.to_period('W').apply(lambda x: x.start_time)
        df_weekly = df_tmp_weekly.groupby('ds').agg({'y': lambda x: x.sum(skipna=False), 
                                                     'yhat_new': lambda x: x.sum(skipna = True), 
                                                     'yhat_upper_new': lambda x: x.sum(skipna = True), 
                                                     'yhat_lower_new': lambda x: x.sum(skipna = True),
                                                    }).reset_index()

        df_weekly.ds = pd.to_datetime(df_weekly.ds)

        if last_day_of_forecast != end_of_last_week:
            df_weekly = df_weekly[df_weekly['ds']<start_of_last_week]

        weekly_median = df_weekly[(df_weekly['ds']>=daily_3_months_ago_future) & (df_weekly.ds <= daily_today)].y.median()
        if weekly_median <= 10:
            if print_info:
                print('too little data for weekly forecast:', weekly_median)
        elif df_weekly[df_weekly.ds <= daily_today].shape[0] < 10:
            if print_info:
                print('too little history observations for weekly forecast:', df_weekly[df_weekly.ds <= daily_today].shape[0])
        else:
            if print_info:
                pplot.one_df(df_weekly.set_index('ds'))
            result_scales['fact_weekly'][counter_id] = df_weekly

In [ ]:
weekly_monthly_predict(final_result_scales['new_predict_with_normalization'].keys(), print_info = False)

In [ ]:
len(counters_for_daily)

In [ ]:
counters_for_daily = counters_for_daily[:len(counters_for_daily) // 2]
counters_weekly = set(final_result_scales['fact_weekly'].keys()) - set(counters_for_daily)
counters_weekly_complex = counters_weekly - set(final_result_daily.keys())
counters_weekly_simple = counters_weekly & set(final_result_daily.keys())

In [ ]:
print(len(counters_for_daily), len(counters_weekly_complex), len(counters_weekly_simple))

#### накладываем условия на weekly

In [ ]:
counters_weekly_complex, counters_weekly_complex_bad_zeros, counters_weekly_complex_bad_median = filter_counters(list(counters_weekly_complex), 'week')
print(len(counters_weekly_complex), len(counters_weekly_complex_bad_zeros), len(counters_weekly_complex_bad_median))

In [ ]:
print(len(counters_for_daily), len(counters_weekly_simple), len(counters_weekly_complex))

In [ ]:
query = '''
PRAGMA AnsiInForEmptyOrNullableItemsCollections;
SELECT
    counter_id
FROM hahn.`home/metrika/data-transfer/prod/mtacs-metrikamain/conv_main_counters`
where status = 'Active';
'''
counters_active_mysql = yql_db.get_df(query=query, token=opts.yql_oauth, add_default_header=True)
counters_active_mysql

In [ ]:
len(counters_for_daily)

In [ ]:
counters_for_daily = list(set(counters_for_daily) & set(counters_active_mysql.counter_id.to_list()))
counters_weekly_complex = list(set(counters_weekly_complex) & set(counters_active_mysql.counter_id.to_list()))
counters_weekly_simple = list(set(counters_weekly_simple) & set(counters_active_mysql.counter_id.to_list()))

In [ ]:
! mkdir counters_three_types_exp

In [ ]:
pd.Series(counters_for_daily).to_csv('counters_three_types_exp/counters_for_daily.csv', index = False, sep = ',')
pd.Series(counters_weekly_complex).to_csv('counters_three_types_exp/counters_weekly_complex.csv', index = False, sep = ',')
pd.Series(counters_weekly_simple).to_csv('counters_three_types_exp/counters_weekly_simple.csv', index = False, sep = ',')

In [ ]:
print(len(counters_for_daily), len(counters_weekly_simple), len(counters_weekly_complex))

In [ ]:
3835 + 4050 + 2591

### Заливаем данные на тестинг

In [ ]:
tvm = get_tvm_client(None)
ticket = tvm.client.get_service_ticket_for('metrika_api_test')

#### заливка

In [ ]:
def prepare_daily_data_for_uploading(counter_id):
    
    global daily_1_year_ago_str
    global daily_today
    
    filename = f'counters_predict/{counter_id}.json'
    
    df_beta = final_result_scales['new_predict_with_normalization'][counter_id][['ds','y','yhat_new','yhat_upper_new','yhat_lower_new']]
    df_beta = df_beta[df_beta['ds']>= daily_1_year_ago_str]
    df_beta['future'] = (df_beta['ds']>daily_today).astype(int)
    df_beta = df_beta.set_index('ds')
    df_beta.loc[daily_today,'future'] = 2
    df_beta.loc[daily_today,'yhat_new'] = df_beta.loc[daily_today,'y']
    df_beta = df_beta.reset_index()
    dates = df_beta.query('future!=1').ds.apply(lambda x: x.strftime('%Y.%m.%d')).to_list()
    values = list(map(int, df_beta.query('future!=1').y.to_list()))
    pred_dates = df_beta.query('future!=0').ds.apply(lambda x: x.strftime('%Y.%m.%d')).to_list()
    pred_values = list(map(int, df_beta.query('future!=0').yhat_new.to_list()))
    pred_lower = list(map(int, df_beta.query('future==1').yhat_lower_new.to_list()))
    pred_upper = list(map(int, df_beta.query('future==1').yhat_upper_new.to_list()))
    recs = {'counter_id': counter_id,
                    'dates':dates,
                    'values':values, 
                    'pred_dates':pred_dates,
                    'pred_values':pred_values,
                    'pred_lower':pred_lower,
                    'pred_upper':pred_upper,
                    "name": "Факт",
                    "pred_name": "План",
                    "lower_name": "Нижняя граница",
                    "upper_name": "Верхняя граница"}
    
    with open(filename, 'w') as f:
        json.dump(recs, f)
    
    return recs

In [ ]:
# ! rm -r counters_predict
# ! mkdir counters_predict

In [ ]:
def prepare_weekly_data_for_uploading(counter_id):
    
    global daily_1_year_ago_str
    global daily_today
    
    filename = f'counters_predict/{counter_id}.json'
    
    df_beta = final_result_scales['fact_weekly'][counter_id][['ds','y','yhat_new','yhat_upper_new','yhat_lower_new']]
    df_beta = df_beta[df_beta['ds']>=daily_1_year_ago_str]
    end_of_today_week = pd.to_datetime(daily_today.to_period('W').end_time.strftime('%Y-%m-%d'))
    if end_of_today_week != daily_today:
        weeks = 1
    else:
        weeks = 0
    df_beta['future'] = (df_beta['ds']>start_of_today_week-timedelta(weeks=weeks)).astype(int)
    df_beta = df_beta.set_index('ds')
    df_beta.loc[start_of_today_week-timedelta(weeks=weeks),'future'] = 2
    df_beta.loc[start_of_today_week-timedelta(weeks=weeks),'yhat_new'] = df_beta.loc[start_of_today_week-timedelta(weeks=weeks),'y']
    df_beta.loc[start_of_today_week-timedelta(weeks=weeks),'yhat_upper_new'] = df_beta.loc[start_of_today_week-timedelta(weeks=weeks),'y']
    df_beta.loc[start_of_today_week-timedelta(weeks=weeks),'yhat_lower_new'] = df_beta.loc[start_of_today_week-timedelta(weeks=weeks),'y']
    
    diff_upper = df_beta.loc[start_of_today_week + pd.Timedelta(weeks = weeks),:]['yhat_upper_new']-df_beta.loc[start_of_today_week + pd.Timedelta(weeks = weeks),:]['yhat_new']
    diff_lower = df_beta.loc[start_of_today_week + pd.Timedelta(weeks = weeks),:]['yhat_new']-df_beta.loc[start_of_today_week + pd.Timedelta(weeks = weeks),:]['yhat_lower_new']
    df_beta.loc[start_of_today_week,'yhat_upper_new'] = df_beta.loc[start_of_today_week,'yhat_new']+diff_upper
    df_beta.loc[start_of_today_week,'yhat_lower_new'] = df_beta.loc[start_of_today_week,'yhat_new']-diff_lower
    
    df_beta = df_beta.clip(0)
    df_beta = df_beta.reset_index()
    #pplot.one_df(df_beta.set_index('ds'))
    dates = df_beta.query('future!=1').ds.apply(lambda x: x.strftime('%Y.%m.%d')).to_list()
    values = list(map(int, df_beta.query('future!=1').y.to_list()))
    pred_dates = df_beta.query('future!=0').ds.apply(lambda x: x.strftime('%Y.%m.%d')).to_list()
    pred_values = list(map(int, df_beta.query('future!=0').yhat_new.to_list()))
    pred_lower = list(map(int, df_beta.query('future!=0').yhat_lower_new.to_list()))
    pred_upper = list(map(int, df_beta.query('future!=0').yhat_upper_new.to_list()))
    recs = {'counter_id': counter_id,
                    'dates':dates,
                    'values':values, 
                    'pred_dates':pred_dates,
                    'pred_values':pred_values,
                    'pred_lower':pred_lower,
                    'pred_upper':pred_upper,
                    "name": "Факт",
                    "pred_name": "План",
                    "lower_name": "Нижняя граница",
                    "upper_name": "Верхняя граница"}
    
    with open(filename, 'w') as f:
        json.dump(recs, f)
    
    return recs

In [ ]:
def push_data(recs, counter_id, urls_uploaded, report_ids):
    
    global today_str 
    
    i = '{i}'
    url_tmpl = 'https://internalapi.test.metrika.yandex.net/internal/v1/counter/{counter_id}/experimental_report/create?type={report}&fromDate={start_date}&toDate={end_date}'
    r = requests.post(
        url=url_tmpl.format(counter_id = counter_id, start_date = today_str, end_date = today_str, report = 'prediction'),
        headers={'X-Ya-Service-Ticket': ticket},
        files = {'file': json.dumps(recs)}, verify=False
    )
    print(r.status_code, r.text[0:300])
    urls_uploaded.append(f'https://semenova2kate-metr-48422.dev.metrika.yandex.ru/experimental/{i}/?id={counter_id}')
    report_ids.append(eval(r.text)['report_id'])

In [ ]:
def upload_data(counters, scale = 'day'):
    
    urls_uploaded = []
    report_ids = []
    for counter_id in counters:
        if scale == 'day':
            recs = prepare_daily_data_for_uploading(counter_id)
        if scale == 'week':
            recs = prepare_weekly_data_for_uploading(counter_id)
        push_data(recs, counter_id, urls_uploaded, report_ids)

    return urls_uploaded, report_ids

In [ ]:
urls_uploaded_daily, report_ids_daily = upload_data(counters_for_daily, 'day')

In [ ]:
urls_uploaded_weekly_complex, report_ids_weekly_complex = upload_data(counters_weekly_complex, 'week')

In [ ]:
urls_uploaded_weekly_simple, report_ids_weekly_simple = upload_data(counters_weekly_simple, 'week')

In [ ]:
def show_uploaded_urls(urls_uploaded, report_ids):
    urls_uploaded_final = []
    i = report_ids[0]
    for url in urls_uploaded:
        print(url.format(i = i))
        urls_uploaded_final.append(url.format(i = i))
        i+=1
    return urls_uploaded_final

In [ ]:
l1 = show_uploaded_urls(urls_uploaded_daily, report_ids_daily)

In [ ]:
l2 = show_uploaded_urls(urls_uploaded_weekly_complex, report_ids_weekly_complex)

In [ ]:
l3 = show_uploaded_urls(urls_uploaded_weekly_simple, report_ids_weekly_simple)

In [ ]:
pd.Series(l1).to_csv('counters_three_types_exp/urls_daily.csv', index = False, sep = ',')
pd.Series(l2).to_csv('counters_three_types_exp/urls_weekly_complex.csv', index = False, sep = ',')
pd.Series(l3).to_csv('counters_three_types_exp/urls_weekly_simple.csv', index = False, sep = ',')

### Дополнительно

#### удаление

In [ ]:
i = report_ids_daily[0]
#i = 6375
for counter_id in counters_for_daily:
        url_tmpl = f'https://internalapi.test.metrika.yandex.net/internal/v1/counter/{counter_id}/experimental_report/{i}'
        r = requests.delete(
            url=url_tmpl,
            headers={'X-Ya-Service-Ticket': ticket},
            verify=False
        )
        print(url_tmpl)
        print(r.text[0:300])
        i+=1

In [ ]:
i = report_ids_weekly_complex[0]
#i = 2725
for counter_id in counters_weekly_complex:
        url_tmpl = f'https://internalapi.test.metrika.yandex.net/internal/v1/counter/{counter_id}/experimental_report/{i}'
        r = requests.delete(
            url=url_tmpl,
            headers={'X-Ya-Service-Ticket': ticket},
            verify=False
        )
        print(url_tmpl)
        print(r.status_code, r.text[0:300])
        i+=1

In [ ]:
i = report_ids_weekly_simple[0]
#i = 971
for counter_id in counters_weekly_simple:
        url_tmpl = f'https://internalapi.test.metrika.yandex.net/internal/v1/counter/{counter_id}/experimental_report/{i}'
        r = requests.delete(
            url=url_tmpl,
            headers={'X-Ya-Service-Ticket': ticket},
            verify=False
        )
        print(url_tmpl)
        print(r.status_code, r.text[0:300])
        i+=1

In [ ]:
q = '''
select * 
from experimental_reports
where type = 'prediction' and status = 'Active'
'''

current_reports = mysql_db.get_df(
    query = q, 
    host = 'metrika-main.haproxy-test.metrika.yandex.net')
current_reports

In [ ]:
i = 30024
for counter_id in current_reports.counter_id.to_list():
        url_tmpl = f'https://internalapi.test.metrika.yandex.net/internal/v1/counter/{counter_id}/experimental_report/{i}'
        r = requests.delete(
            url=url_tmpl,
            headers={'X-Ya-Service-Ticket': ticket},
            verify=False
        )
        print(url_tmpl)
        print(r.status_code, r.text[0:300])
        i+=1

### сохранение данных

#### сохраняем urls на yt

In [ ]:
urls_uploaded_total = l1 + l2 + l3

In [ ]:
import re

In [ ]:
df_urls_to_yt = pd.DataFrame({'counter_id':list(map(lambda x: int(re.findall(r'id=(\d+)', x)[0]), urls_uploaded_total)),
                'url':urls_uploaded_total})

In [ ]:
today_str

In [ ]:
yt_db.create_folder(f'//home/metrica-analytics/garskova-d/predictions/{today_str}')
yt_db.create_folder(f'//home/metrica-analytics/garskova-d/predictions/{today_str}/result')

In [ ]:
yt_db.write_to_yt(f'//home/metrica-analytics/garskova-d/predictions/{today_str}/result/urls', 
                 df_urls_to_yt,
                 append = False)

##### сохрянаем залитые инфо на yt

In [ ]:
q = '''
select * 
from experimental_reports
where type = 'prediction' and status = 'Active'
'''

loaded_reports_df = mysql_db.get_df(
    query = q, 
    host = 'metrika-main.haproxy-test.metrika.yandex.net')

In [ ]:
yt_db.write_to_yt(f'//home/metrica-analytics/garskova-d/predictions/{today_str}/mysql_and_info', 
                 loaded_reports_df[['report_id','counter_id','file_name']],
                 append = False)

##### Сохраняем залитые предикты на yt

In [ ]:
yt_db.create_folder(f'//home/metrica-analytics/garskova-d/predictions/{today_str}/daily_fact')

In [ ]:
path_predicts = 'counters_predict'

from os import listdir
from os.path import isfile, join
import re
onlyfiles = [f for f in listdir(path_predicts) if isfile(join(path_predicts, f))]

In [ ]:
df_tmp = pd.DataFrame()

for path in tqdm.tqdm(onlyfiles):
    with open(f'counters_predict/{path}') as json_file:
        data = json.load(json_file)
    df_tmp = df_tmp.append(pd.DataFrame({'counter_id':data['counter_id'],'rec':str(data)}, index = [0]))
df_tmp = df_tmp.reset_index(drop = True)

yt_db.write_to_yt(f'//home/metrica-analytics/garskova-d/predictions/{today_str}/result/counters_result_recs', 
             df_tmp,
             append = False)

##### сохраняем daily_fact на yt

In [ ]:
path_predicts = 'counters_daily_fact'

onlyfiles = [f for f in listdir(path_predicts) if isfile(join(path_predicts, f))]

df_daily_facts_to_yt = pd.DataFrame()
for path in tqdm.tqdm(onlyfiles):
    data = pd.read_csv(f'{path_predicts}/{path}').set_index('ds').to_json()
    counter_id = int(re.findall(r'\d+', path)[0])
    df_tmp = pd.DataFrame({'counter_id':counter_id,'df':data}, index = [0])
    df_daily_facts_to_yt = df_daily_facts_to_yt.append(df_tmp)
df_daily_facts_to_yt = df_daily_facts_to_yt.reset_index(drop = True)

In [ ]:
yt_db.write_to_yt(f'//home/metrica-analytics/garskova-d/predictions/{today_str}/daily_fact/counters_daily_fact_df', 
                 df_daily_facts_to_yt,
                 append = False)

##### сохраняем тип счетчика из экспа на yt

In [ ]:
pd.Series(counters_for_daily).to_csv('counters_three_types_exp/counters_for_daily.csv', index = False, sep = ',')
pd.Series(counters_weekly_complex).to_csv('counters_three_types_exp/counters_weekly_complex.csv', index = False, sep = ',')
pd.Series(counters_weekly_simple).to_csv('counters_three_types_exp/counters_weekly_simple.csv', index = False, sep = ',')

In [ ]:
df1 = pd.DataFrame(pd.Series(counters_for_daily), columns = ['counter_id'])
df1['type'] = 'daily'
df2 = pd.DataFrame(pd.Series(counters_weekly_complex), columns = ['counter_id'])
df2['type'] = 'weekly_complex'
df3 = pd.DataFrame(pd.Series(counters_weekly_simple), columns = ['counter_id'])
df3['type'] = 'weekly_simple'

In [ ]:
df_types = df1.append(df2).append(df3).reset_index(drop = True)

In [ ]:
yt_db.write_to_yt(f'//home/metrica-analytics/garskova-d/predictions/{today_str}/counters_exp_type', 
                 df_types,
                 append = False)

In [ ]:
query = f'''

insert into hahn.`home/metrica-analytics/garskova-d/predictions/{today_str}/visits` with truncate

SELECT
    counter_id,
    probable_host,
    main_country_traff,
    tier,
    source,
    visits_last_month,
    users_last_month,
    is_rsya,
    client_id,
    clicks_visits,
FROM hahn.`//home/metrica-analytics/reports/metrika_marketing_cube/marketing_cube_final/m/latest`
'''
run = yql_db.get_df(query=query, token=opts.yql_oauth, add_default_header=True)
run

In [ ]:
today_str

In [ ]:
query = f'''
INSERT INTO hahn.`home/metrica-analytics/garskova-d/predictions/{today_str}/result/urls_and_visits` WITH TRUNCATE 

select t.*,
        '((' || url || ' link))' as url_wiki
 from (

SELECT
    *
FROM hahn.`home/metrica-analytics/garskova-d/predictions/{today_str}/result/urls` as a

left join 
(select * from 
hahn.`home/metrica-analytics/garskova-d/predictions/{today_str}/visits`) as b

on a.counter_id = b.counter_id

left join

(select * from 
hahn.`home/metrica-analytics/garskova-d/predictions/{today_str}/counters_exp_type`) as c

on a.counter_id = c.counter_id) as t

order by visits_last_month desc;
'''
run = yql_db.get_df(query=query, token=opts.yql_oauth, add_default_header=True)
run

### загрузка предиктов с yt

In [ ]:
#today_str = '2022-05-21'
query = f'''
PRAGMA AnsiInForEmptyOrNullableItemsCollections;
SELECT *
FROM hahn.`//home/metrica-analytics/garskova-d/predictions/{today_str}/result/counters_result_recs`
;
'''
df_from_yql = yql_db.get_df(query=query, token=opts.yql_oauth, add_default_header=True)
df_from_yql

In [ ]:
df_from_yql = df_from_yql[df_from_yql.counter_id.isin(counters_active_mysql.counter_id.to_list())]

In [ ]:
df_from_yql.rec.values[0]

#### Заливаем данные с yt

In [ ]:
def push_data_from_yt(df_from_yql):
    
    global today_str 
    
    urls_uploaded = []
    
    for el in df_from_yql.iterrows():
        counter_id = el[1].counter_id
        recs = eval(el[1].rec)
    
        url_tmpl = 'https://internalapi.test.metrika.yandex.net/internal/v1/counter/{counter_id}/experimental_report/create?type={report}&fromDate={start_date}&toDate={end_date}'
        r = requests.post(
            url=url_tmpl.format(counter_id = counter_id, start_date = today_str, end_date = today_str, report = 'prediction'),
            headers={'X-Ya-Service-Ticket': ticket},
            files = {'file': json.dumps(recs)}, verify=False
        )

        i = eval(r.text)['report_id']
        print(r.status_code, r.text[0:300])
        urls_uploaded.append(f'https://semenova2kate-metr-48422.dev.metrika.yandex.ru/experimental/{i}/?id={counter_id}')

    return urls_uploaded

In [ ]:
urls_uploaded_total = push_data_from_yt(df_from_yql)

In [ ]:
pd.DataFrame(urls_uploaded_total)

In [ ]:
pd.DataFrame(urls_uploaded_total).values[1]

In [ ]:
pd.DataFrame(urls_uploaded_total).to_csv('urls_uploaded_total_26_07_2022.csv', index = False, sep = ',')

In [ ]:
df_from_yql[df_from_yql['counter_id']==65746927]

In [ ]:
65746927

In [ ]:
df_from_yql.iloc[:813]

#### удаляем, если что-то пошло не так

In [ ]:
#i = report_ids_daily[0]
i = 18881
for counter_id in df_from_yql.counter_id.to_list():
        url_tmpl = f'https://internalapi.test.metrika.yandex.net/internal/v1/counter/{counter_id}/experimental_report/{i}'
        r = requests.delete(
            url=url_tmpl,
            headers={'X-Ya-Service-Ticket': ticket},
            verify=False
        )
        print(url_tmpl)
        print(r.status_code, r.text[0:300])
        i+=1

### Графички вручную

In [ ]:
pplot.one_df(final_result_scales['fact_weekly'][6596923][['ds','y','yhat_new','yhat_upper_new','yhat_lower_new']].set_index('ds'))

In [ ]:
pplot.one_df(final_result_scales['new_predict_with_normalization'][66791488][['ds','y','yhat_new','yhat_upper_new','yhat_lower_new']].set_index('ds'))

### Archive

#### get

In [ ]:
# http://6xboyh55epm4o674.sas.yp-c.yandex.net:8080/internal/v1/counter/57008650/experimental_reports

In [ ]:
# /internal/v1/counter/{counterId}/experimental_report/{report_id}

In [ ]:
# for counter_id in list(final_result_daily.keys()):
#     url_tmpl = f'https://internalapi.test.metrika.yandex.net/internal/v1/counter/{counter_id}/experimental_report/1210'
#     r = requests.get(
#         url=url_tmpl,
#         headers={'X-Ya-Service-Ticket': ticket},
#         verify=False
#     )
#     print(url_tmpl)
#     print(r.text)
#     i+=1

#### old counters selector

In [ ]:
# query = f'''
# select a.CounterID as CounterID from 

# (select distinct CounterID
# from (SELECT CounterID, StartDate as ds, sum(Sign) as y
# FROM visits_all
# WHERE StartDate <= toDate('{today_str}')
#         and ClientIPNetwork != 'YANDEX_SERVER' 
#         and IsRobot = 0
#         and CounterID global in (select CounterID from garskova.active_counters_in_interface)
# group by CounterID, StartDate)
# group by CounterID) as a

# inner join 

# (SELECT CounterID, sum(Sign) as y
# FROM visits_all
# WHERE StartDate >= toDate('{today_str}') - 30 and StartDate <= toDate('{today_str}')
#         and ClientIPNetwork != 'YANDEX_SERVER' 
#         and IsRobot = 0
#         and CounterID global in (select CounterID from garskova.active_counters_in_interface)
# group by CounterID
# having y > 0) as b

# on a.CounterID = b.CounterID

# inner join

# (SELECT CounterID, sum(Sign) as y
# FROM visits_all
# WHERE StartDate <= toDate('{daily_end_date_str}')
#         and ClientIPNetwork != 'YANDEX_SERVER' 
#         and IsRobot = 0
#         and CounterID global in (select CounterID from garskova.active_counters_in_interface)
# group by CounterID
# having y > 0) as c

# on a.CounterID = c.CounterID

# FORMAT TabSeparatedWithNames
# '''
# df_counters_sample = ch.get_df(query)
# df_counters_sample
# #counters = df_counters_sample.CounterID.tolist()